# PDF와 JSONL 파일을 읽어서 같이 RAG로 할용할 수 있습니다.

In [29]:
import fitz
import re 
import json 
from langchain_core.documents import Document
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [30]:
pdf_file_paths = [
    "/workspace/거버넌스.pdf",
]

# 예시에서는 jsonl은 question과 answer로 구성되어있습니다.
jsonl_file_paths = [
    "/workspace/2025-AI-Challeng-finance/cybersecurity_data_regex_cleaned.jsonl",
]

# PDF 처리

In [31]:
def extract_text_from_pdf(pdf_path:str) -> str : 
    try :
        with fitz.open(pdf_path) as doc : 
            return "".join(page.get_text() for page in doc)
    except Exception as e : 
        print(e)
        return ""

def chunk_general_pdf(text:str, source:str) -> list[Document] : 
    if not text :
        return []

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=126,
        separators=["\n\n", "\n"]
    ) 

    # text를 text_splitter를 통해서 잘게 분리합니다.
    chuncks = text_splitter.split_text(text)

    documents = []
    # Document 형식으로 변환 해주어야지 vector DB 저장이 가능합니다.
    # chunck들을 page_content에 넣고, 추가 metadata를 넣어줍니다.
    for i, chunk in enumerate(chuncks) : 
        doc = Document(
            page_content = chunk,
            metadata = {"source":source, "chunk_num":i+1, "data_type":"general_pdf"}
        )
        documents.append(doc)
    return documents



# JSONL 처리

In [35]:
def chunck_QA_jsonl(text:str, all_documents:list[Document]) -> list[Document] : 
    if not text : 
        return []
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=126
    )

    split_docs = text_splitter.split_text(text)
    print(f"{len(split_docs)}개의 'Q&A' 청크를 추가했습니다.")

    for doc in split_docs:
        doc.metadata["data_type"] = "qna_jsonl"
    all_documents.extend(doc)

    
    return doc

In [36]:
all_documents = []

print(f'=======PDF 데이터를 처리합니다======')

for pdf_path in pdf_file_paths:
    print(f"{pdf_path}처리중........")
    text = extract_text_from_pdf(pdf_path)
    if not text : 
        print(f"{pdf_path}에는 아무런 text가 존재하지 않아 건너뜁니다.🦘")
        continue

    documents = chunk_general_pdf(text, pdf_path)
    all_documents.extend(documents)
    print(f"{pdf_path}파일 정리가 완료되었습니다 => {len(documents)}개의 청크 생성완료.")


for jsonl_path in jsonl_file_paths:
    print(f"{jsonl_path}처리중........")
    try:
        loader = JSONLoader(
            file_path=jsonl_path,
            # 예시 jsonl이 {"question" : , "answer" : } 형식이기 때문에 다음과 같이 사용
            jq_schema='"질문: " + .question + "\\n답변: " + .answer',
            json_lines=True
        )
        documents_from_jsonl = loader.load()
        doc = chunck_QA_jsonl(documents_from_jsonl, all_documents)

    except Exception as e : 
        print(e)


=======PDF 데이터를 처리합니다======
/workspace/거버넌스.pdf처리중........
/workspace/거버넌스.pdf파일 정리가 완료되었습니다 => 7개의 청크 생성완료.
/workspace/2025-AI-Challeng-finance/cybersecurity_data_regex_cleaned.jsonl처리중........
expected string or bytes-like object, got 'list'


In [37]:
print("\n--- ✅ 최종 통합 완료 ---")
print(f"모든 파일로부터 총 {len(all_documents)}개의 Document를 생성했습니다.")

if all_documents:
    general_pdf_sample = next((doc for doc in all_documents if doc.metadata.get("data_type") == "general_pdf"), None)
    if general_pdf_sample:
        print("\n📑 일반 PDF에서 온 데이터 샘플:")
        print(general_pdf_sample)
    
    jsonl_sample = next((doc for doc in all_documents if doc.metadata.get("data_type") == "qna_jsonl"), None)
    if jsonl_sample:
        print("\n📝 JSONL에서 온 데이터 샘플:")
        print(jsonl_sample)


--- ✅ 최종 통합 완료 ---
모든 파일로부터 총 7개의 Document를 생성했습니다.

📑 일반 PDF에서 온 데이터 샘플:
page_content='거버넌스
거버넌스(governance)는 일반적으로 ‘과거의 일방적인 정부 주도적 경향에서 벗어나 정부, 기업, 비정부
기구 등 다양한 행위자가 공동의 관심사에 대한 네트워크를 구축하여 문제를 해결하는 새로운 국정운영
의 방식’을 말한다.[1] 그렇지만 다양한 학문 분야에서 서로 다른 맥락으로 쓰이고 있어, 아직 정의에 대한
명확한 학문적 합의는 이루어지지 않았다고 볼 수 있다.[2]
거버넌스는 일반적으로, 그리고 학문적으로 빈번히 사용됨에도 불구하고, 그 다양한 의미로 인해 정의가
쉽지 않다. 이는 다양한 학문 분야에서 서로 다른 맥락으로 사용되고 있기 때문이다.[3]
존 피에르(Jon Pierre)와 피터스(B. Guy Peters)는 “정책 결정에 있어 정부 주도의 통제와 관리에서 벗어나
다양한 이해당사자가 주체적인 행위자로 협의와 합의 과정을 통하여 정책을 결정하고 집행해 나가는 사
회적 통치 시스템”으로 정의했다.[4]' metadata={'source': '/workspace/거버넌스.pdf', 'chunk_num': 1, 'data_type': 'general_pdf'}
